## 1. Carrega as bases de dados

In [ ]:
#!pip install openai --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [ ]:
import pandas as pd

# Modelos
MODELO = 'text-embedding-3-large'

# Pasta com os dados
PASTA_DADOS = './dados/'

# Pasta com os dados de jurisprudência já tratados
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Pasta onde serão armazenados os resultados desse caderno
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/6_gera_embeddings_js_open_ai/{MODELO}/'

# Tamanho do lote
#QUANTIDADE_DE_ITENS_CARREGADOS = 5
SOBRESCREVER_EMBEDDINGS = False
TAMANHO_DO_LOTE = 512

# Função que carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)

    return doc

In [ ]:
from formatador import remove_html

# Carrega os arquivos para doc
doc = carrega_juris_tcu()
#doc = doc.head(QUANTIDADE_DE_ITENS_CARREGADOS)

#Remove tags do enunciado
doc['ENUNCIADO'] = doc['ENUNCIADO'].apply(remove_html)

#Transforma dataframe em dicionário
doc = doc.to_dict(orient='list')
print(doc.keys())

## 2. Obtenção dos embeddings.

In [ ]:
# Função para dividir dicionário em lotes
def dividir_dicionario_em_lotes(dicionario, tamanho_do_lote):
    vetor_de_dicionarios = []
    max_len = max(len(v) for v in dicionario.values())  # Encontrando o vetor de valores mais longo
    
    for i in range(0, max_len, tamanho_do_lote):
        novo_dicionario = {chave: valores[i:i + tamanho_do_lote] for chave, valores in dicionario.items()}
        vetor_de_dicionarios.append(novo_dicionario)
    
    return vetor_de_dicionarios

In [ ]:
import numpy as np
import torch

# Função para reconstruir dicionário a partir de lotes
def reconstruir_dicionario_a_partir_de_lotes(vetor_de_dicionarios):
    dicionario_reconstruido = {}
    
    # Inicializando listas vazias para cada chave no primeiro dicionário do vetor
    for chave in vetor_de_dicionarios[0].keys():
        dicionario_reconstruido[chave] = []
    
    # Iterando sobre cada dicionário no vetor e concatenando os valores para cada chave
    for dicionario_lote in vetor_de_dicionarios:
        for chave, valores in dicionario_lote.items():
            dicionario_reconstruido[chave].extend(valores)
    
    # Transforma numpy array em tensor
    dicionario_reconstruido['mean_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['mean_hidden_state']))
    
    return dicionario_reconstruido

In [ ]:
# Função para converter dicionário em dataframe
def dicionario_para_dataframe(dicionario):
    df = pd.DataFrame(dicionario)
    return df

In [ ]:
# Função para converter dataframe em dicionario
def dataframe_para_dicionario(df):
    return df.to_dict(orient='list')

In [ ]:
import openai
from getpass import getpass

openai.api_key = getpass('Qual a chave da OpenAI?')

# Função para extrair embeddings
def get_embedding(texto, model=MODELO):
    texto = texto.replace("\n", " ")
    return openai.embeddings.create(input = [texto], model=model).data[0].embedding

In [ ]:
# Processar e salvar embeddings
import pickle
from tqdm import tqdm
import os

# Divide doc_encoded em lotes
doc_em_lotes = dividir_dicionario_em_lotes(doc, TAMANHO_DO_LOTE)

# Processa e salva embeddings
for i, dicionario in enumerate(tqdm(doc_em_lotes), start=1):
    
    caminho_arquivo = f'{PASTA_RESULTADO_CADERNO}{MODELO}_embeddings_js_{i}.pickle'
    if  not SOBRESCREVER_EMBEDDINGS and os.path.exists(caminho_arquivo):
        continue

    dicionario_df = dicionario_para_dataframe(dicionario) 
    dicionario_df['mean_hidden_state'] = dicionario_df.ENUNCIADO.apply(lambda x: get_embedding(x, model=MODELO))
    
    # Cria estrutura que será salva em arquivo
    embeddings_js = {
        'key': dicionario_df['KEY'].tolist(),
        'mean_hidden_state': dicionario_df['mean_hidden_state'].tolist()
    }
    
    # Gravando lote em um arquivo .pickle
    with open(caminho_arquivo, 'wb') as arquivo_pickle:
        pickle.dump(embeddings_js, arquivo_pickle)

## 4. Cálculo da distância entre os embeddings.

In [ ]:
# Função para restaurar embeddings dos arquivos pickle

import os
import pickle

def restaurar_doc_encoded_de_pickle(pasta_resultado_caderno):
    # Lista para armazenar os dicionários lidos dos arquivos .pickle
    doc_encoded_restaurado = []

    # Listando todos os arquivos .pickle no diretório especificado
    arquivos_pickle = [arq for arq in os.listdir(pasta_resultado_caderno) if arq.endswith('.pickle')]

    # Ordenando os arquivos pelo número (assumindo que os nomes dos arquivos seguem o padrão embeddings_js_X.pickle)
    arquivos_pickle.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Lendo cada arquivo .pickle e restaurando o dicionário
    for nome_arquivo in arquivos_pickle:
        caminho_arquivo = os.path.join(pasta_resultado_caderno, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo_pickle:
            dicionario_restaurado = pickle.load(arquivo_pickle)
            doc_encoded_restaurado.append(dicionario_restaurado)

    return reconstruir_dicionario_a_partir_de_lotes(doc_encoded_restaurado)

In [ ]:
doc_encoded_restaurado = restaurar_doc_encoded_de_pickle(PASTA_RESULTADO_CADERNO)

In [ ]:
import numpy as np

doc_hidden = doc.copy()
doc_hidden['mean_hidden_state'] = doc_encoded_restaurado['mean_hidden_state']
print(f"mean_hidden_state: {doc_hidden['mean_hidden_state'].size()}")

In [ ]:
doc

In [ ]:
# Extraindo os embeddings de duas frases
embedding1_tensor = doc_hidden['mean_hidden_state'][0]
embedding2_tensor = doc_hidden['mean_hidden_state'][1]

# Normalizando os embeddings
embedding1_norm = embedding1_tensor / embedding1_tensor.norm()
embedding2_norm = embedding2_tensor / embedding2_tensor.norm()

# Calculando a similaridade por cosseno
cosine_similarity = torch.dot(embedding1_norm, embedding2_norm)

print(f"Similaridade por cosseno: {cosine_similarity.item()}")

In [ ]:
doc_encoded_restaurqado

In [ ]:
print(doc_hidden['ENUNCIADO'][0])
print(doc_hidden['ENUNCIADO'][1])
print(doc_hidden['ENUNCIADO'][4])